## AIHUB Preprocessing

<br>

### <b>목차 </b>

<br>

#### <b>1) 라이브러리 </b>

<br>

#### <b>2) 데이터 전처리 유형 </b>

<b>유형 1</b>
<br>공통 전처리 코드에서 KSS 1번
<br>나머지 데이터셋

<b>유형 2</b>
<br>공통 전처리 코드에서 KSS 3번
<br><요약문 및 레포트 생성 데이터>, <대규모 구매도서 기반 한국어 말뭉치 데이터>, <도서자료 요약>

<b>유형 3</b>
<br>추가사항
<br>- 구두점 ". . .", ". .", ".."을 온점 "."으로 교체
<br><웹데이터 기반 한국어 말뭉치 데이터>

<b>유형  4</b>
<br>추가사항
<br>- 숫자.(1. 2. ...), 원숫자(①, ②, ③, ..., ⑭, ⑮)를 기준으로 문장 분리. 그 이후 KSS 실행
<br><법률 규정 (판결서 약관 등) 텍스트 분석 데이터>

<b>유형 5</b>
<br>추가사항
<br>- '제n조의n항(text)', '제n조(text)' 꼴 제거
<br>- 숫자.(1. 2. ...), 원숫자(①, ②, ③, ..., ⑭, ⑮)를 기준으로 문장 분리
<br>- 문장의 시작이 특수문자인 문장(원문자 - ①, ②, ③, ..., ⑭, ⑮ 제외) 제외
<br><전문분야 말뭉치>의 법령 및 판례 텍스트

<b>유형 6</b>
<br>추가사항
<br>- KSS를 사용하지 않고 HTML 언어('br', 'tr', 'td', 'table', 'tbody', 'rowspan')를 기준으로 문장 분리
<br><행정 문서 대상 기계독해 데이터>의 ##_tableqa.json

<br>

#### <b>1) 라이브러리 </b>

<br>

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
import re
import kss
import json
from mecab import MeCab

<br>

<b>KSS(한국어 문장 분리기)</b>

<br>Github: https://github.com/hyunwoongko/kss

<br>오픈소스 라이브러리 버전: 3.7.3

<br>라이센스: 3-Clause BSD License(BSD-3-Clause)

<br>작동 방식:
<br>KSS의 함수 split_sentences의 argument에는 ‘use_heuristic (bool)’, ‘backend (str)’ 등이 있다.
<br>‘use_heuristic (bool)'에서 False를 설정하면, 문장부호를 기준으로 문장 분리를 한다(Punctuation-only segmentation). 문법을 준수하는 공식적인 텍스트(wiki, news, essays)에 맞는 문장분리 방식이다.
<br>C++가 작동할 수 있는 작업환경이면, 형태소분석기 Mecab을 KSS의 ‘backend (str)’로  사용하여 KSS 라이브러리 실행 속도를 높인다.

<br>

<br>

### <b>2) 데이터 전처리 유형 </b>

<br>

### 유형 1

공통 전처리 코드에서 KSS 1번 실행

<br>

<요약문 및 레포트 생성 데이터>, <대규모 구매도서 기반 한국어 말뭉치 데이터>, <도서자료 요약>
<br>
<br>를 제외한 데이터셋은 모두 KSS 1번만 한다.

<br>

구어체가 존재하지 않는 텍스트에서는 KSS 3번의 효과가 없으므로 KSS 1번만 한다.

신문기사 인용문의 경우 <"~합니다."고 밝혔다.>에서 "~합니다"는 구어체가 아니라
<br>신문기사에 맞게 편집된 문어체이므로
<br>신문기사에서는 KSS 1번 실행한다. 

<br>

In [ ]:
def formal_preprocessing_text_kss1(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
    # 사례: "김가람 기자, 현재 상황 알려주시죠.[리포트] 네, 중계차가 나와있는 이곳 서귀포시 법환포구에는 저녁 들어서 바람이 더 강해지고 있습니다."

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 사례: "국가 전체적으로도 자원의 낭비 혹은 왜곡된 자원배분을 가져온다고 보았다(최진욱, 2006, p.10에서 재인용)."

        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        # 사례: "핑크베리의  ...  불렀다. (또한 유명한 휴대전화 블랙베리가 있다.) 핑크베리사는 ... 있다.

    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
    # 사례: "가. 의회 복지건설위원장 의정활동 및 직무수행과 관련된 소요경비를 지출한다."
    # 사례: "나. (생략) 한다."
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거
            # 사례: "교육부장관 서남수] "예 고민해 보겠습니다."


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)
  
    return preprocessing_sentence_list

<br>

### 유형 2

<br>

공통 전처리 코드에서 KSS 3번 실행

<br>

<요약문 및 레포트 생성 데이터>, <대규모 구매도서 기반 한국어 말뭉치 데이터>, <도서자료 요약>
<br>

이 3개의 데이터셋을 제외한 나머지 말뭉치 데이터셋에서는 구어체가 나오지 않는다.

<br>

구어체가 포함된 특정 텍스트
<br>(문학, 인터뷰, 속기사가 기록한 글(회의, 행사 등))에서는
<br>KSS 3번 반복에 대한 효과가 있다

<br>

In [ ]:
def formal_preprocessing_text_kss3(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
    # 사례: "김가람 기자, 현재 상황 알려주시죠.[리포트] 네, 중계차가 나와있는 이곳 서귀포시 법환포구에는 저녁 들어서 바람이 더 강해지고 있습니다."

    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 사례: "국가 전체적으로도 자원의 낭비 혹은 왜곡된 자원배분을 가져온다고 보았다(최진욱, 2006, p.10에서 재인용)."

        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        # 사례: "핑크베리의  ...  불렀다. (또한 유명한 휴대전화 블랙베리가 있다.) 핑크베리사는 ... 있다.

    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
    # 사례: "가. 의회 복지건설위원장 의정활동 및 직무수행과 관련된 소요경비를 지출한다."
    # 사례: "나. (생략) 한다."

    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거
            # 사례: "교육부장관 서남수] "예 고민해 보겠습니다."


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

<br>

### 유형 3

<br>

<웹데이터 기반 한국어 말뭉치 데이터>

추가사항 - 구두점 ". . .", ". .", ".."을 온점 "."으로 교체

<br>

<웹데이터 기반 한국어 말뭉치 데이터>에서는 구두점이 이상한 문장이 있다.

In [11]:
sentences = ["리모콘을 pc의 마우스처럼 상하좌우로 움직여서 보고 싶은 콘텐츠를 클릭하면 된다.. .",
"지난 달 말 수율 문제 등을 (이름) 애플이 아이폰8에 일체형 지문센서를 탑재할지 아직 결정하지 못했다는 보도가 나왔었다..",
"개발이 완료된 상황에서 선도기업 위상을 놓칠 수 없는 까닭이다.. . . ."]
for sentence in sentences:
    print(sentence)

리모콘을 pc의 마우스처럼 상하좌우로 움직여서 보고 싶은 콘텐츠를 클릭하면 된다.. .
지난 달 말 수율 문제 등을 (이름) 애플이 아이폰8에 일체형 지문센서를 탑재할지 아직 결정하지 못했다는 보도가 나왔었다..
개발이 완료된 상황에서 선도기업 위상을 놓칠 수 없는 까닭이다.. . . .


<br>

따라서 위의 함수 formal_preprocessing_text(source)에서
<br>구두점에 관련된 코드를 추가한다.
<br>이렇게 구두점에 관련된 코드를 추가하는 것이 <웹데이터 기반 한국어 말뭉치 데이터>의 차이점이다.

<br>

In [ ]:
sentence = sentence.replace(". . .", ".")
sentence = sentence.replace(". .", ".")
sentence = sentence.replace("..", ".")
# 구두점 ". . .", ". .", ".."을 온점 "."으로 교체

<br>

### 유형 4

<br>

<법률 규정 (판결서 약관 등) 텍스트 분석 데이터>

추가사항 - 숫자.(1. 2. ...), 원숫자(①, ②, ③, ..., ⑭, ⑮)를 기준으로 문장 분리. 그 이후 KSS 실행

<br>

<법률 규정 (판결서 약관 등) 텍스트 분석 데이터>에는 단락 중간에 원숫자가 존재하여 KSS의 문장 분리 성능을 저해한다.
<br>따라서 KSS 이전에 숫자.(1. 2. ...), 원숫자(①, ②, ③, ..., ⑭, ⑮)로 문장을 분리하는 작업이 필요하다.

In [1]:
paragraph = '제6조 (이용계약의 변경 및 조정)\n① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. 다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.'
print(paragraph)

제6조 (이용계약의 변경 및 조정)
① 이용기간은 13박 14일(2주)를 기본으로 하되, 계약을 체결하는 때에 사업자와 이용자가 상호 협의하여 기간을 조정할 수 있습니다. 다만, 사업자는 고객이 13박 14일보다 단기의 이용을 요청한다는 이유로 계약의 체결을 거절할 수 없습니다.


In [ ]:
sources = re.split(r"[0-9]+[.]|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮", source)
# 숫자.(1. 2. ...), 원숫자(①, ②, ③, ...)를 기준으로 문장 분리

<br>

### 유형 5

<br>

<전문분야 말뭉치>의 법령 및 판례 텍스트

KSS 이전에 법령 및 판례 관련 전처리 작업을 선행한다.

<br>

<전문분야 말뭉치>의 법령 및 판례 텍스트에서는
<br>KSS 실행 이후에 다음 작업을 거쳐야 문장이 제대로 분리될 수 있다.
<br><br>1) '제n조의n항(text)', '제n조(text)' 꼴 제거
<br>2) 숫자.(1. 2. ...), 원숫자(①, ②, ③, ...)를 기준으로 문장 분리
<br>3) 문장의 시작이 특수문자인 문장(원문자 - ①, ②, ③, ..., ⑭, ⑮ 제외) 제외

<br>

다른 말뭉치의 법률 텍스트에서는 이러한 작업이 필요하지 않다.
<br><전문분야 말뭉치>의 법령 및 판례 텍스트에서만 필요한 작업이다.

<br>

In [7]:
print("제1조(설치 및 기능)① 행정 각 부ㆍ처ㆍ청 간의 협조를 긴밀하게 하며 국무회의에 제출된 의안과 국무회의로부터 지시받은 사항을 심의하기 위하여 차관회의를 둔다. ② 차관회의는 국무에 관하여 국무회의에 건의할 수 있다. [전문개정 2011.11.7]")

제1조(설치 및 기능)① 행정 각 부ㆍ처ㆍ청 간의 협조를 긴밀하게 하며 국무회의에 제출된 의안과 국무회의로부터 지시받은 사항을 심의하기 위하여 차관회의를 둔다. ② 차관회의는 국무에 관하여 국무회의에 건의할 수 있다. [전문개정 2011.11.7]


<br>

In [ ]:
def legal_preprocessing_text(source):
    preprocessing_sentence_list = []
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
    # 사례: "김가람 기자, 현재 상황 알려주시죠.[리포트] 네, 중계차가 나와있는 이곳 서귀포시 법환포구에는 저녁 들어서 바람이 더 강해지고 있습니다."
    
    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 사례: "국가 전체적으로도 자원의 낭비 혹은 왜곡된 자원배분을 가져온다고 보았다(최진욱, 2006, p.10에서 재인용)."

        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        # 사례: "핑크베리의  ...  불렀다. (또한 유명한 휴대전화 블랙베리가 있다.) 핑크베리사는 ... 있다.

        
    
    if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', source[:2])) == True:
        source = source.replace(source[:2], "")
        
    source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)
    # '가.', '나.', ... 형태의 문자열 제거 
    # 사례: "가. 의회 복지건설위원장 의정활동 및 직무수행과 관련된 소요경비를 지출한다."
    # 사례: "나. (생략) 한다."
    
    source = re.sub(r'제\d{1,}조의\d{1,}\([^)]+\)', "", source)
    source = re.sub(r'제\d{1,}조\([^)]+\)', "",source)
    sources = re.split(r"[0-9]+[.]|①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮", source)
    # '제n조의n항(text)', '제n조(text)' 꼴 제거
    # 숫자.(1. 2. ...), 원숫자(①, ②, ③, ...)를 기준으로 문장 분리
    # 사례 : "제1조(설치 및 기능)① 행정 (생략) 차관회의를 둔다. ② 차관회의는 (생략)  있다. [전문개정 2011.11.7]"
    
    for source in sources:
    
        for sentence in kss.split_sentences(source, use_heuristic=False,
                                            num_workers=32):
        # KSS(Korean Sentence Segmentation)로 문장 분리 
        # Formal articles (wiki, news, essays): recommend to False

            if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮]", sentence[0]) is not None and \
                bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
                len(sentence.split()) > 5:
                # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
                # 원문자 - ①, ②, ③, ④, ⑤, ⑥, ⑦, ⑧, ⑨, ⑩, ⑪, ⑫, ⑬, ⑭, ⑮ 제외) 제외
                # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
                # 다섯 어절 이하 문장 제외

                sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
                # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

                sentence = sentence.strip()
                # strip으로 앞뒤 공백 제거

                total_length = len(sentence.replace(" " , ""))
                hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
                hangeul_ratio = hangeul_length / total_length
                if hangeul_ratio >= 0.5:
                # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                    preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list

<br>

### 유형 6

<br>

<행정 문서 대상 기계독해 데이터>의 ##_tableqa.json

HTML 언어가 포함되어 있어 KSS 작업을 하지 않는다.

<br>

HTML 언어 'br', 'tr', 'td', 'table', 'tbody', 'rowspan' 특수기호 '<', '>', '/'가 포함된 텍스트에서는
<br>KSS가 효과가 없다.

<br>

Markdown으로 <행정 문서 대상 기계독해 데이터>의 원천데이터를 입력하면 아래의 표와 같이 출력된다.
<br>HTML 언어가 포함되었기 때문이다.

<br>

"1. 기획담당관-10318(2021.6.18.)호 및 공원여가과-2651(2021.6.21.)호, 공공개발기획단<br>   -5300(2021.6.22.)호와 관련입니다.<br>2. 귀 부서에서 신청하신 간행물 발간등록번호를 아래와 같이 통보해드리니, 안내에 따라 조치하신 후 간행물 납본결과를 작성하여 담당자 이메일로 제출해주시기 바랍니다.<br>  가. 간행물 발간등록번호 부여결과 <table><tbody><tr><td>신청부서</td><td>간행물명</td><td> 발간등록번호</td></tr><tr><td>기획조정실기획담당관</td><td>2021 하반기 달라지는 서울생활</td><td>51********-*******10</td></tr><tr><td>서부공원녹지사업소공원여가과</td><td>공원의 기억 월드컵공원(영문판)</td><td>51********-*******01</td></tr><tr><td>공공개발기획단</td><td>공공부지 전략거점개발계획수립</td><td>51********-*******01</td></tr></tbody></table> 간행물 발간등록번호 부여결과"

In [5]:
print("1. 기획담당관-10318(2021.6.18.)호 및 공원여가과-2651(2021.6.21.)호, 공공개발기획단<br>   -5300(2021.6.22.)호와 관련입니다.<br>2. 귀 부서에서 신청하신 간행물 발간등록번호를 아래와 같이 통보해드리니, 안내에 따라 조치하신 후 간행물 납본결과를 작성하여 담당자 이메일로 제출해주시기 바랍니다.<br>  가. 간행물 발간등록번호 부여결과 <table><tbody><tr><td>신청부서</td><td>간행물명</td><td> 발간등록번호</td></tr><tr><td>기획조정실기획담당관</td><td>2021 하반기 달라지는 서울생활</td><td>51********-*******10</td></tr><tr><td>서부공원녹지사업소공원여가과</td><td>공원의 기억 월드컵공원(영문판)</td><td>51********-*******01</td></tr><tr><td>공공개발기획단</td><td>공공부지 전략거점개발계획수립</td><td>51********-*******01</td></tr></tbody></table> 간행물 발간등록번호 부여결과")

1. 기획담당관-10318(2021.6.18.)호 및 공원여가과-2651(2021.6.21.)호, 공공개발기획단<br>   -5300(2021.6.22.)호와 관련입니다.<br>2. 귀 부서에서 신청하신 간행물 발간등록번호를 아래와 같이 통보해드리니, 안내에 따라 조치하신 후 간행물 납본결과를 작성하여 담당자 이메일로 제출해주시기 바랍니다.<br>  가. 간행물 발간등록번호 부여결과 <table><tbody><tr><td>신청부서</td><td>간행물명</td><td> 발간등록번호</td></tr><tr><td>기획조정실기획담당관</td><td>2021 하반기 달라지는 서울생활</td><td>51********-*******10</td></tr><tr><td>서부공원녹지사업소공원여가과</td><td>공원의 기억 월드컵공원(영문판)</td><td>51********-*******01</td></tr><tr><td>공공개발기획단</td><td>공공부지 전략거점개발계획수립</td><td>51********-*******01</td></tr></tbody></table> 간행물 발간등록번호 부여결과


<br>

In [ ]:
def tableqa_preprocessing_text(source):
    
    preprocessing_sentence_list = []
    source_split = re.split('br|tr|td|table|tbody|rowspan', source)
    # 문자열 'br', 'tr', 'td', 'table', 'tbody', 'rowspan'을 기준으로 문장 분리

    for source in source_split:
        sentence = re.sub('(br|tr|td|table|tbody|rowspan|<|>|/)', '', source)
        # 문자열 'br', 'tr', 'td', 'table', 'tbody', 'rowspan', 특수기호 '<', '>', '/'를 제외
            
        sentence = sentence.strip()
        # strip으로 앞뒤 공백 제거
        
        sentence = re.sub(r"\[.*?\]|\{.*?\}", "", sentence)
        # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거
        # 사례: [그림 2-2], .[리포트] 
        # 사례: "김가람 기자, 현재 상황 알려주시죠.[리포트] 네, 중계차가 나와있는 이곳 서귀포시 법환포구에는 저녁 들어서 바람이 더 강해지고 있습니다."
        
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(sentence)

        try:
            bracket_form = re.compile('\(([^)]+)')
            text_in_small_bracket = bracket_form.findall(sentence)


            if type(text_in_small_bracket) == str:

                text = text_in_small_bracket

                text_size = len(text)
                last_index = sentence.find(text) + len(text)
                if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                    sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    sentence = sentence.replace(small_bracket, text)    

            elif type(text_in_small_bracket) == list:

                for text in text_in_small_bracket:

                    text_size = len(text)
                    last_index = sentence.find(text) + len(text)
                    if len(sentence) >= last_index+1 and sentence[last_index-text_size-1] == '(' and sentence[last_index+1] == '.':
                        sentence = sentence.replace(sentence[last_index-text_size-1 : last_index+1] + ".", ".")

                    if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                        small_bracket = "(" + text + ")"
                        sentence = sentence.replace(small_bracket, text)    

        except:
            pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 사례: "국가 전체적으로도 자원의 낭비 혹은 왜곡된 자원배분을 가져온다고 보았다(최진욱, 2006, p.10에서 재인용)."

        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        # 사례: "핑크베리의  ...  불렀다. (또한 유명한 휴대전화 블랙베리가 있다.) 핑크베리사는 ... 있다.

    
        if bool(re.match(r'[가나다라마바사아자차카타파하]+[.]', sentence[:2])) == True:
            sentence = source.replace(source[:2], "")

        sentence = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", sentence)
        # '가.', '나.', ... 형태의 문자열 제거 
        # 사례: "가. 의회 복지건설위원장 의정활동 및 직무수행과 관련된 소요경비를 지출한다."
        # 사례: "나. (생략) 한다."    

        if len(sentence) > 1 and \
        re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
        bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and  \
        len(sentence.split()) > 5:
        # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, +, 
        # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
        # 다섯 어절 이하 문장 제외
        
            
            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거
            
            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외
                preprocessing_sentence_list.append(sentence)

    return preprocessing_sentence_list